# 训练

In [ ]:
from dl.lstm import LSTMAdvanced
from statics.Statics import Statics
import numpy as np
import pandas as pd

lstm = LSTMAdvanced(feature_type=0)

lstm.dataset_construct()
lstm.train()
lstm.draw_loss()

# 性能

原文：包括对模型预测的不确定性 $^a$ /精确度 $^b$ 的估计，以及对模型性能的衡量 $^c$

#### a: 不确定性

蒙特卡洛 Dropout（MC Dropout）

In [15]:
# MC 采样  
mean, std = lstm.monte_carlo_sampling(lstm.model, lstm.x_test)  

# 计算预测区间  
lower_bound = mean - 1.96 * std  # 95% 置信区间下界  
upper_bound = mean + 1.96 * std  # 95% 置信区间上界  

# 打印结果  
print("Mean prediction:", mean)  
print("Uncertainty (std):", std)  
print("95% confidence interval:", lower_bound, upper_bound) 

Mean prediction: 5.970329
Uncertainty (std): 0.54119897
95% confidence interval: 4.909579 7.031079


#### b: 精确度

这个我搜了下，好像主要还是指下面那几个误差

#### c: 模型性能

测试集上计算

Mean Squared Error (MSE): 4.3301

Root Mean Squared Error (RMSE): 2.0809

Mean Absolute Error (MAE): 2.0297

R² Score: -3.3301

In [14]:
model = lstm.model

X_test = lstm.x_test
y_test = lstm.y_test

X_train = lstm.x_train
y_train = lstm.y_train

import torch  
import torch.nn as nn  
import numpy as np  
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(y_true, y_pred):  
    """  
    计算模型的回归评价指标  
    :param y_true: 真实值 (numpy array)  
    :param y_pred: 预测值 (numpy array)  
    :return: None  
    """  
    mse = mean_squared_error(y_true, y_pred)  
    rmse = np.sqrt(mse)  
    mae = mean_absolute_error(y_true, y_pred)  
    r2 = r2_score(y_true, y_pred)  
    
    print(f"Mean Squared Error (MSE): {mse:.4f}")  
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")  
    print(f"Mean Absolute Error (MAE): {mae:.4f}")  
    print(f"R² Score: {r2:.4f}")
    
# 假设 test_loader 是测试集的 DataLoader  
model.eval()  # 设置模型为评估模式  

with torch.no_grad():  # 禁用梯度计算  
    y_train_pred = model(X_train)
    y_pred = model(X_test)

# 计算评价指标  
y_train_pred = lstm.transform_from_tensor_data(y_train_pred)
y_train = lstm.transform_from_tensor_data(y_train)
print("---------- Train Set Evaluation ----------")
evaluate_model(y_train, y_train_pred)

y_pred = lstm.transform_from_tensor_data(y_pred)
y_test = lstm.transform_from_tensor_data(y_test)
print("---------- Test Set Evaluation ----------")
evaluate_model(y_test, y_pred)

s = Statics()
lstm.draw_cmp(sample_index=s.query_country('USA'), title='USA Gold Metal Prediction vs. True Values')

---------- Train Set Evaluation ----------
Mean Squared Error (MSE): 0.9836
Root Mean Squared Error (RMSE): 0.9918
Mean Absolute Error (MAE): 0.7571
R² Score: 0.9844
---------- Test Set Evaluation ----------
Mean Squared Error (MSE): 4.3301
Root Mean Squared Error (RMSE): 2.0809
Mean Absolute Error (MAE): 2.0297
R² Score: -3.3301


# 预测

生成预测奖牌榜文件

In [ ]:
lstm.input_future_construct()
y = lstm.model(lstm.x_input_future)
lstm.get_medal_board(output=y, year=2028, type='pred')